## Creating a script to parse blast files of custom or standard tabular format

In [68]:
# function for parsing blast file
def old_blast_parser(blastfile, tab=None):
    """parse tabular blast files to retrieve all information for downstream"""
    blast = open(blastfile)
    if tab is None: #'standard': # alternative add later
        std = {'qseqid':0, 'sseqid':1, 'pident':2, 'length':3, 'mismatch':4, 'gapopen':5,'qstart':6, 'qend':7, 'sstart':8, 'send':9, 'evalue':10, 'bitscore':11}
        fmt_list = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
    #else:
    integers = ['qlen','slen','qstart','qend','sstart','send','length','nident','mismatch','positive','gapopen','gaps','qcovs','qcovhsp']    
    floaters = ['bitscore','score','pident','ppos']
    ranges = {}
    rng_index = {}
    #len_min = 100
    #min_pct = 30
    hit_index = {}
    hit_dict = {}
    hitlist = []
    fld_range = len(fmt_list)
    field_dict = {}
    for f in range(0,fld_range):
        field_dict[fmt_list[f]]=f
        
    ct = 0
    for hit in blast:
        hit = hit.strip('\n')
        parts = hit.split('\t')
        ## have to change this to be flexible as to input format, start with standard format
        
        for fld in range(0,fld_range):
            fld_name = fmt_list[fld]
            fld_value = field_dict[fld_name]
            ranges.setdefault(hitnum, {})['qseqid']=query
            
        query = parts[std['qseqid']]
        #print(query)
        subject = parts[std['sseqid']]
        pct_ident = float(parts[std['pident']])
        aln_length = int(parts[std['length']])
        num_mismatch = int(parts[std['mismatch']])
        gap_open = int(parts[std['gapopen']])
        query_start = int(parts[std['qstart']])
        query_end = int(parts[std['qend']])
        substart = int(parts[std['sstart']])
        subend = int(parts[std['send']])
        e_val = parts[std['evalue']]
        bit_score = parts[std['bitscore']]
        #if pct_ident > min_pct: # filtering will happen outside script
        if query in hitlist:
            ct += 1
        else:
            hitlist.append(query)
            ct = 0
        hitnum = query+';hit'+str(ct)
        rng_index.setdefault(query, []).append(hitnum)
        ranges.setdefault(hitnum, {})['qseqid']=query
        ranges.setdefault(hitnum, {})['sseqid']=subject
        ranges.setdefault(hitnum, {})['pident']=pct_ident
        ranges.setdefault(hitnum, {})['length']=aln_length
        ranges.setdefault(hitnum, {})['mismatch']=num_mismatch
        ranges.setdefault(hitnum, {})['gapopen']=gap_open
        ranges.setdefault(hitnum, {})['qstart']=query_start
        ranges.setdefault(hitnum, {})['qend']=query_end
        ranges.setdefault(hitnum, {})['sstart']=substart
        ranges.setdefault(hitnum, {})['send']=subend
        ranges.setdefault(hitnum, {})['evalue']=e_val
        ranges.setdefault(hitnum, {})['bitscore']=bit_score
    return(rng_index, ranges)


In [35]:
# function for parsing blast file
def blast_parser(blastfile, tab='standard'):
    """parse tabular blast files to retrieve all information for downstream"""
    blast = open(blastfile)
    if tab is 'standard': #'standard': # alternative add later 
        fmt_list = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
    else:
        fmt_list = tab
    integers = ['qlen','slen','qstart','qend','sstart','send','length','nident','mismatch','positive','gapopen','gaps','qcovs','qcovhsp']    
    floaters = ['bitscore','score','pident','ppos','evalue']
    ranges = {}
    rng_index = {}
    hit_index = {}
    hit_dict = {}
    hitlist = []
    fld_range = len(fmt_list)
    ct = 0
    for hit in blast:
        hit = hit.strip('\n')
        parts = hit.split('\t')
        for fld in range(0,fld_range):
            fld_name = fmt_list[fld]
            # differentiate between field types
            if fld_name in integers:
                fld_value = int(parts[fld])
            elif fld_name in floaters:
                fld_value = float(parts[fld])
            else:
                fld_value = parts[fld]    
            if fld == 0:    
                if fld_value in hitlist:
                    ct += 1
                else:
                    hitlist.append(fld_value)
                    ct = 0
                hitnum = str(fld_value)+';hit'+str(ct)
                rng_index.setdefault(str(fld_value), []).append(hitnum)

            ranges.setdefault(hitnum, {})[fld_name]=fld_value
            

    return(rng_index, ranges)


In [30]:
std_fmt = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']

In [17]:
blast_results = blast_parser('Prim_rel_up_blastp')

In [36]:
# try with different format
blast_results = blast_parser('Prim_rel_up_blastp', std_fmt)

In [37]:
blast_results[0]

{'MA_1001785_MA_1001785.g230.t1': ['MA_1001785_MA_1001785.g230.t1;hit0'],
 'MA_101067_MA_101067.g84.t1': ['MA_101067_MA_101067.g84.t1;hit0'],
 'MA_101067_MA_101067.g85.t1': ['MA_101067_MA_101067.g85.t1;hit0'],
 'MA_101067_MA_101067.g86.t1': ['MA_101067_MA_101067.g86.t1;hit0'],
 'MA_101067_MA_101067.g87.t1': ['MA_101067_MA_101067.g87.t1;hit0'],
 'MA_10145917_MA_10145917.g232.t1': ['MA_10145917_MA_10145917.g232.t1;hit0'],
 'MA_10251013_MA_10251013.g231.t1': ['MA_10251013_MA_10251013.g231.t1;hit0'],
 'MA_102538_MA_102538.g100.t1': ['MA_102538_MA_102538.g100.t1;hit0'],
 'MA_102538_MA_102538.g97.t1': ['MA_102538_MA_102538.g97.t1;hit0'],
 'MA_102538_MA_102538.g98.t1': ['MA_102538_MA_102538.g98.t1;hit0'],
 'MA_102538_MA_102538.g99.t1': ['MA_102538_MA_102538.g99.t1;hit0',
  'MA_102538_MA_102538.g99.t1;hit1'],
 'MA_10265000_MA_10265000.g202.t1': ['MA_10265000_MA_10265000.g202.t1;hit0'],
 'MA_10337257_MA_10337257.g243.t1': ['MA_10337257_MA_10337257.g243.t1;hit0'],
 'MA_10383303_MA_10383303.g242.

In [32]:
indx = blast_results[1]
indx

{'MA_1001785_MA_1001785.g230.t1;hit0': {'bitscore': 30.4,
  'evalue': 4.6,
  'gapopen': 2,
  'length': 71,
  'mismatch': 46,
  'pident': 32.394,
  'qend': 89,
  'qseqid': 'MA_1001785_MA_1001785.g230.t1',
  'qstart': 19,
  'send': 338,
  'sseqid': 'gi|74753219|sp|Q9UBB6.1|NCDN_HUMAN',
  'sstart': 270},
 'MA_101067_MA_101067.g84.t1;hit0': {'bitscore': 33.1,
  'evalue': 6.3,
  'gapopen': 2,
  'length': 94,
  'mismatch': 64,
  'pident': 29.787,
  'qend': 390,
  'qseqid': 'MA_101067_MA_101067.g84.t1',
  'qstart': 297,
  'send': 1627,
  'sseqid': 'gi|160358869|sp|Q02224.2|CENPE_HUMAN',
  'sstart': 1536},
 'MA_101067_MA_101067.g85.t1;hit0': {'bitscore': 34.3,
  'evalue': 3.0,
  'gapopen': 10,
  'length': 303,
  'mismatch': 177,
  'pident': 21.452,
  'qend': 310,
  'qseqid': 'MA_101067_MA_101067.g85.t1',
  'qstart': 19,
  'send': 383,
  'sseqid': 'gi|73619888|sp|Q973E7.1|CCA_SULTO',
  'sstart': 131},
 'MA_101067_MA_101067.g86.t1;hit0': {'bitscore': 582.0,
  'evalue': 0.0,
  'gapopen': 1,
  'le

In [72]:
# quickly, get swissprot names
swisslist = []
for k,v in indx.iteritems():
    full_id = v['sseqid']
    swiss = full_id.split('|')[-1]
    swisslist.append(swiss)
len(swisslist)

216

In [71]:
len(indx)

216

In [38]:
# write script to filter blast_parser output to file as table
def get_parameters(parameter_file):
    para_file = open(parameter_file, 'r')
    para_dict = {}
    for line in para_file:
        line = line.strip('\n')
        if line.startswith('#'):
            continue
        else:
            line_parts = line.split('=')
            para_name = line_parts[0]
            para_value =line_parts[1]
            if para_name == 'FORMAT':
                fieldlist = para_value.split(' ')
                para_dict['FORMAT']=fieldlist
            elif para_name == 'MIN_LENGTH':
                para_dict['MIN_LENGTH']=int(para_value)
            elif para_name == 'MIN_PCT_ID':
                para_dict['MIN_PCT_ID']=int(para_value)
    
    
    return para_dict

In [39]:
paras1 = get_parameters('parameters_file_template.txt')
paras1

{'FORMAT': ['qseqid',
  'sseqid',
  'pident',
  'length',
  'mismatch',
  'gapopen',
  'qstart',
  'qend',
  'sstart',
  'send',
  'evalue',
  'bitscore'],
 'MIN_LENGTH': 100,
 'MIN_PCT_ID': 50}

In [40]:
fmt1 = paras1['FORMAT']
fmt1

['qseqid',
 'sseqid',
 'pident',
 'length',
 'mismatch',
 'gapopen',
 'qstart',
 'qend',
 'sstart',
 'send',
 'evalue',
 'bitscore']

In [41]:
# retry with new formatting
blast_results2 = blast_parser('Prim_rel_up_blastp', fmt1)

In [42]:
blast_results2[1]

{'MA_1001785_MA_1001785.g230.t1;hit0': {'bitscore': 30.4,
  'evalue': 4.6,
  'gapopen': 2,
  'length': 71,
  'mismatch': 46,
  'pident': 32.394,
  'qend': 89,
  'qseqid': 'MA_1001785_MA_1001785.g230.t1',
  'qstart': 19,
  'send': 338,
  'sseqid': 'gi|74753219|sp|Q9UBB6.1|NCDN_HUMAN',
  'sstart': 270},
 'MA_101067_MA_101067.g84.t1;hit0': {'bitscore': 33.1,
  'evalue': 6.3,
  'gapopen': 2,
  'length': 94,
  'mismatch': 64,
  'pident': 29.787,
  'qend': 390,
  'qseqid': 'MA_101067_MA_101067.g84.t1',
  'qstart': 297,
  'send': 1627,
  'sseqid': 'gi|160358869|sp|Q02224.2|CENPE_HUMAN',
  'sstart': 1536},
 'MA_101067_MA_101067.g85.t1;hit0': {'bitscore': 34.3,
  'evalue': 3.0,
  'gapopen': 10,
  'length': 303,
  'mismatch': 177,
  'pident': 21.452,
  'qend': 310,
  'qseqid': 'MA_101067_MA_101067.g85.t1',
  'qstart': 19,
  'send': 383,
  'sseqid': 'gi|73619888|sp|Q973E7.1|CCA_SULTO',
  'sstart': 131},
 'MA_101067_MA_101067.g86.t1;hit0': {'bitscore': 582.0,
  'evalue': 0.0,
  'gapopen': 1,
  'le